In [13]:
import csv
from pathlib import Path
import cv2
import imageio
import imageio_ffmpeg
import numpy as np
import onnxruntime as ort
import torch
import os
import torchvision.transforms as T
from PIL import Image
from sparrow_datums import BoxTracking
from tqdm import tqdm
from torch.utils.data import DataLoader
from speed_trapv3.keypoints.dataset import SegmentationDataset
from speed_trapv3.keypoints.utils import Holdout
import matplotlib.pyplot as plt
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.keypoints.config import Config
import matplotlib.pyplot as plt



In [14]:
data = SegmentationDataset(Holdout.TEST)
test_dataloader = DataLoader(data, batch_size=1, shuffle=False)

In [15]:
model = SegmentationModel().eval().cuda()
model.load(Config.trained_model_path)

<All keys matched successfully>

In [16]:
overall_rel_error = 0
count = 0
for batch in iter(test_dataloader):
    x = batch['image']
    x = x.cuda()
    result = model(x)
    relative_error = torch.norm(
        batch["keypoints"].cuda() - result["keypoints"].cuda()
    ) / torch.norm(batch["keypoints"].cuda())
    overall_rel_error += relative_error
    count += 1


In [17]:
overall_rel_error = overall_rel_error.detach().cpu().numpy()/ count

In [18]:
print(f"The relative error of the test set is {overall_rel_error * 100}%.")
print("Relative error is the ratio of the magnitude of the difference between groundtruth and prediction compared to the magnitude of the ground truth.")
print("That means that for every groundtruth with a magnitude of 100 units, there is an error of 20 units in the corresponding prediction.")

The relative error of the test set is 20.193108759428323%
Relative error is the ratio of the magnitude of the difference between groundtruth and prediction compared to the magnitude of the ground truth.
That means that for every groundtruth with a magnitude of 100 units, there is an error of 20 units in the corresponding prediction.
